In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(tracks) {
    return false;
}

In [ ]:
#Notes signed and dated by KD are debugging, any unsigned notes are from the BVH lab as part of the orignal 
#script -KD 2023.08.03

import matplotlib.pyplot as plt
import lumicks.pylake as lk
import itertools
import ipywidgets
import numpy as np
import glob
import os
import shutil
# We use skimage to downsample the data
from skimage.measure import block_reduce
from statistics import mean
from statistics import median
from scipy.stats import sem
from matplotlib_venn import venn2 #this track is used for making venn diagrams of interactions
#if you get an error here just add in the terminal from the python environment: pip intall matplotlib-venn
%matplotlib inline
# Use notebook if you're in Jupyter Notebook
%matplotlib widget

filename = glob.glob('*.h5') #fetches filename assuming only one .h5 in notebook
if len(filename) == 1:
    print (filename[0])
else: 
    print ("too many or too few .h5 files")
    print(len(filename))
    
#adjusted original command in track 1 of this cell from filename = glob.glob('*.h5') to filename = glob.glob('**/*.h5')
#so glob.glob can find the h5 files in subdirectory -KD 2023.08.03

#adjusted this back temporarily but is a good trick! -- MS 2023-08-16

plt.close('all')#This closes any plots left open to save memory and loads up the .h5 file in folder
file = lk.File(str(filename[0]))
print(lk.File(str(filename[0])))
list(file.kymos)

In [ ]:
forcex = file["Force HF"]["Force 1x"] #this cell downsamples the force to save memory and make it easier to visualize

# time traces (seconds)
time = forcex.timestamps/1e9
time = time - time[0]
sample_rate = forcex.sample_rate

downsampled_rate = 50 # Hz, this rate can be changed as needed

# downsample the force, nanostage position and time
forcex_downsamp = forcex.downsampled_by(int(sample_rate/downsampled_rate))
time_downsamp = forcex_downsamp.timestamps/1e9
time_downsamp = time_downsamp - time_downsamp[0]

median_force = (median(forcex_downsamp.data))
kymowidget = lk.KymoWidgetGreedy
#added here to try bypassing the above error from pylake wiki guessing probably an
#effect of depreciations -KD 2023.08.03

In [ ]:
list(file.kymos)

#added for debugging below KD 2023.08.03

kymosname = [x for x in file.kymos.keys()]

#added with Sawyer's help where kymosname should be an object that matches list(file.kymos) as output from above
#but is a hashable type, prevents error I was getting in the next cell where kymosname has been placed -KD 2023.08.04

_, kymo = file.kymos.popitem() #after defining the downsample rate earlier, this actually does the downsampling

data = file.kymos[kymosname[0]].get_image(channel="red")

#note this depends on the collection channel -- for Cy3 we use green -og comment on script
#adjusted value from file.kymos["11"] to value recognized, see cell above to use for debugging -KD 2023.08.04

downsample_factor = 1
data = block_reduce(data, (1, downsample_factor))
#rgb = kymo.rgb_image
kymo = file.kymos[kymosname[0]]

In [ ]:
# define a color adjustment so we can see the data better. this will preview the dataset in all three channels and also show the force.
adjustment = lk.ColorAdjustment(0, 0.1)


name, kymo = file.kymos.popitem()

kymo.plot_with_force("1x", "rgb", adjustment=adjustment, aspect_ratio=0.5)
plt.show()

pixel_size = kymo.pixelsize_um
pixel_size_int = pixel_size[0]

In [ ]:
lk.KymoWidgetGreedy(kymo, 'rgb', aspect='auto', adjustment=adjustment,  min_length=4, pixel_threshold=5, window=7, sigma=1, vmax=2)
plt.show()
#this first plot is to look at the kymograph on the pixel scale in order to decide if an upper and lower bound need to be applied

In [ ]:
crop_top = 4.79
crop_bottom = 20.16
#renamed these variables so that things run smoothly downstream -- MAS 9-6-23
adjusted_lower_bound = crop_top
adjusted_upper_bound = crop_bottom
steps_between_bounds = int((adjusted_upper_bound - adjusted_lower_bound) * 10) #defines number of 100 nm steps between bounds
downsample_factor = 1

kymo_crop = file.kymos[kymosname[0]].crop_by_distance(crop_top, crop_bottom)

#viewing the kymographs this way will show them after they are cropped
lk.KymoWidgetGreedy(kymo_crop, 'rgb', aspect='auto', adjustment=adjustment,  min_length=4, pixel_threshold=2, window=7, sigma=1, vmax=2)


In [ ]:
#This cell for tracking tracks with kymowidget greedy. It opens two views -- one with a fixed aspect ratio and one adjustable
#By saving and loading the tracks as redkymotracks.txt, they can easily be swapped back and forth between views as needed. Also used a scaling color map. MAS 9-6-23
red_tracks = lk.KymoWidgetGreedy(kymo_crop.downsampled_by(4), "red", aspect="auto", min_length=4, pixel_threshold=1, window=12, sigma=0.25, vmax=25, correct_origin=True, cmap='viridis')

#I prefer the auto aspect so I just commented this one out for now.
#kymowidget1 = lk.KymoWidgetGreedy(kymo_crop.downsampled_by(downsample_factor), "red", axis_aspect_ratio=2, min_length=4, pixel_threshold=1, window=7, sigma=0.25, vmax=2, correct_origin=True)

In [ ]:
#NOTE: MUST HAVE KYMOTRACKS LOADED OR THIS WILL ERROR.
#Used this to replace the tracks and save_tracks functions below that have since been replaced with the 
#KymoTrackGroup tracks. Printed values should match, if they do not the analysis between the two is not the same.
#Be sure to load and save between the widgets, but ultimately kymowidget1, the 2nd one, is the one that I've
#chosen to export the .csv output required for further steps. The comma delimiter for the .csv is highly
#recommended as comma delimited .csv files can be opened with Excel -KD 2023.08.07


red_tracks.save_tracks('red-ontarget_kymotracks.csv', delimiter = ',')


In [ ]:
#This cell for tracking green tracks with kymowidget greedy. It opens two views -- one with a fixed aspect ratio and one adjustable
#By saving and loading the tracks as redkymotracks.txt, they can easily be swapped back and forth between views as needed. Also used a scaling color map. MAS 9-6-23
green_tracks = lk.KymoWidgetGreedy(kymo_crop.downsampled_by(4), "green", aspect="auto", min_length=4, pixel_threshold=1, window=12, sigma=0.25, vmax=25, correct_origin=True, cmap='viridis')

#I prefer the auto aspect so I just commented this one out for now.
#kymowidget1 = lk.KymoWidgetGreedy(kymo_crop.downsampled_by(downsample_factor), "red", axis_aspect_ratio=2, min_length=4, pixel_threshold=1, window=7, sigma=0.25, vmax=2, correct_origin=True)

In [ ]:
green_tracks.save_tracks('green-ontarget_kymotracks.txt', delimiter = ',')

In [ ]:
#Replaced the above two obsolete cells with these commands containing updated, non-depreciated
#parameters -KD 2023.08.07

#Displays number of tracks tracked and number of theoretical bins that should be used. -MAS 2024.02.27
numberofredtracks=len(red_tracks.tracks)
numberofgreentracks=len(green_tracks.tracks)
print(numberofredtracks)
print(numberofgreentracks)
numberofredbins = int(round(np.sqrt (numberofredtracks)) )
print (numberofredbins)
numberofgreenbins = int(round(np.sqrt (numberofgreentracks)) )

In [ ]:
longest_track_idx = np.argmax([len(track) for track in red_tracks.tracks])  # Get the index of the longest track
longest_track = red_tracks.tracks[longest_track_idx]

plt.figure()
plt.plot(longest_track.seconds, longest_track.position)
plt.xlabel("Time [s]")
plt.ylabel("Position [$\mu$m]")
plt.show()

In [ ]:
#red trackS SECTION--------------------------------------------------------------
redlengths = (track.time_idx[-1]-track.time_idx[0] for track in red_tracks.tracks)
# Get the index of the longest kymo track
redlongest_index = np.argmax(redlengths)

# Select the longest red track
redlongest_track = red_tracks.tracks[redlongest_index]
dt = kymo.line_time_seconds
redtimes = np.array([(track.time_idx[-1]-track.time_idx[0]) for track in red_tracks.tracks])*dt*downsample_factor

#this section plots the positions of the longest track and also shows the raw image of that track 
#also leaves 1 second empty on each side so the start and end can be seen.
plt.figure(figsize=(5, 3))
plt.plot(np.array(redlongest_track.time_idx) * dt, np.array(redlongest_track.coordinate_idx ) * dt, color = 'red') 
plt.xlabel('Time [s]')
plt.ylabel('Position [$\mu$m]')
plt.tight_layout()
plt.show()

plt.figure(figsize=(7, 4))

kymo.plot('rgb', aspect="auto", adjustment=adjustment)
plt.xlim([(np.array(redlongest_track.time_idx[0])) * dt -1, (np.array(redlongest_track.time_idx[-1])*dt +1 )])
plt.ylim(adjusted_upper_bound+1, adjusted_lower_bound-1) 
plt.show()

In [ ]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

# Create a list to store track indices and average photon counts for all red tracks
avg_counts_data = []
figures = []

downsample_factor = 1

for i, track in enumerate(red_tracks.tracks):
    # Get time and photon count data
    time_vals = np.array(track.time_idx) * dt
    photon_counts = track.sample_from_image(3)

    # Apply downsampling
    time_vals = time_vals[::downsample_factor]
    photon_counts = photon_counts[::downsample_factor]

    # Calculate line of best fit
    coeffs = np.polyfit(time_vals, photon_counts, 1)  # Linear fit (degree 1)
    best_fit_line = np.poly1d(coeffs)

    # Calculate the average y-value
    avg_y = np.mean(photon_counts)
    
    # Store the data for saving in "index, value" format
    avg_counts_data.append(f"{i}, {avg_y:.2f}")

    # Create figure for each plot
    fig, ax = plt.subplots(figsize=(6, 5))
    ax.plot(time_vals, photon_counts, label=f'Track {i}', color='red')
    ax.plot(time_vals, best_fit_line(time_vals), '--', color='black', label='Best Fit')
    ax.set_ylabel('Photon count')
    ax.set_xlabel('Time [s]')
    ax.set_title(f'Photon count for Track {i}\n(Average: {avg_y:.2f})')
    ax.set_ylim(0, 100)
    ax.legend()
    plt.tight_layout()
    
    # Store figure in list
    figures.append(fig)
    
    # Print the average photon count value
    print(f'Track {i}: Average photon count = {avg_y:.2f}')

    # Close the individual figure after saving
    plt.close(fig)  # Close individual figure to free up memory

# Save the average photon counts to a text file
with open("average_red_photon_counts.txt", "w") as file:
    file.write("\n".join(avg_counts_data))

print("Average photon counts saved to average_red_photon_counts.txt")

# Save all figures in a single image file
num_plots = len(figures)
cols = 4  # Number of columns in the grid
rows = (num_plots + cols - 1) // cols  # Compute rows dynamically

fig, axes = plt.subplots(rows, cols, figsize=(cols * 6, rows * 5))
axes = axes.flatten() if num_plots > 1 else [axes]

for ax in axes[len(figures):]:
    ax.axis('off')  # Turn off unused subplots

# Create the large figure with all the subplots and save it
for i, fig in enumerate(figures):
    fig.canvas.draw()
    img_array = np.array(fig.canvas.renderer.buffer_rgba())
    axes[i].imshow(img_array)
    axes[i].axis('off')
    axes[i].set_title(f'Track {i}')

plt.tight_layout()
plt.savefig("all_red_photon_counts.png", dpi=300)

# Close the main figure that contains all subplots
plt.close(fig)

print("All plots saved to all_red_photon_counts.png")

In [ ]:
#Trialling out segmenting the data to fit a line of best fit at different intervals 

# Create a list to store track indices and segment-wise averages
avg_counts_data = []
figures = []

downsample_factor = 1  # Adjust downsampling if needed
segment_duration = 20  # Fit a new line every 20 seconds

for i, track in enumerate(red_tracks.tracks):
    # Get time and photon count data
    time_vals = np.array(track.time_idx) * dt
    photon_counts = np.array(track.sample_from_image(3, correct_origin=True))  # Fix warning

    # Apply downsampling
    time_vals = time_vals[::downsample_factor]
    photon_counts = photon_counts[::downsample_factor]

    # Ensure they are NumPy arrays
    time_vals = np.array(time_vals)
    photon_counts = np.array(photon_counts)

    # Determine time bins for segmentation
    start_time = time_vals[0]
    end_time = time_vals[-1]
    time_bins = np.arange(start_time, end_time, segment_duration)

    segment_averages = []

    # Create figure
    fig, ax = plt.subplots(figsize=(6, 5))
    ax.plot(time_vals, photon_counts, '-', color='red', label=f'Track {i}')  # Full kymotrack

    # Iterate over each time segment
    for j in range(len(time_bins) - 1):
        # Select points within the current time window
        mask = (time_vals >= time_bins[j]) & (time_vals < time_bins[j + 1])
        segment_time = time_vals[mask]
        segment_counts = photon_counts[mask]  # ✅ Fix boolean indexing issue

        if len(segment_time) > 1:  # Ensure enough data points exist
            # Fit a line for the segment
            coeffs = np.polyfit(segment_time, segment_counts, 1)
            best_fit_line = np.poly1d(coeffs)

            # Compute segment average
            avg_y = np.mean(segment_counts)
            segment_averages.append(f"{i}, {time_bins[j]:.1f}-{time_bins[j+1]:.1f}, {avg_y:.2f}")

            # Plot best-fit line for this segment
            ax.plot(segment_time, best_fit_line(segment_time), '--', label=f'Fit {j}')

    # Store segment-wise averages
    avg_counts_data.extend(segment_averages)

    # Finalize figure
    ax.set_ylabel('Photon count')
    ax.set_xlabel('Time [s]')
    ax.set_title(f'Photon count for Track {i}')
    ax.set_ylim(0, max(photon_counts) * 1.1)
    ax.legend()
    plt.tight_layout()
    
    # Store figure in list
    figures.append(fig)

    # Close figure to free memory
    plt.close(fig)

# Save segment-wise average photon counts to a text file
with open("segment_average_red_photon_counts.txt", "w") as file:
    file.write("\n".join(avg_counts_data))

print("Segment-wise average photon counts saved to segment_average_red_photon_counts.txt")

# Save all figures in a single image file
num_plots = len(figures)
cols = 4
rows = (num_plots + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(cols * 6, rows * 5))
axes = axes.flatten() if num_plots > 1 else [axes]

for ax in axes[len(figures):]:
    ax.axis('off')  # Hide unused subplots

for i, fig in enumerate(figures):
    fig.canvas.draw()
    img_array = np.array(fig.canvas.renderer.buffer_rgba())
    axes[i].imshow(img_array)
    axes[i].axis('off')
    axes[i].set_title(f'Track {i}')

plt.tight_layout()
plt.savefig("all_segmented_red_photon_counts.png", dpi=300)
plt.close(fig)

print("All segmented plots saved to all_segmented_red_photon_counts.png")

In [ ]:
#THIS CELL FOR PROCESSING red CHANNEL DATA
#to plot photon count along the longest track
plt.figure(figsize=(6, 5))
plt.plot( np.array(redlongest_track.time_idx) * dt, redlongest_track.sample_from_image(3), color='red') #this samples 3 pixels away from track position
plt.ylabel('Photon count')
plt.xlabel('Time [s]')
plt.title('Photon counts along the longest track')
plt.tight_layout()
plt.show()


#plots track length as a histogram. numberofbins was defined earlier as square root of n
plt.figure(figsize=(5, 3))
plt.hist(redtimes, numberofredbins, color='red')
plt.ylabel('Events')
plt.xlabel('Duration (s)')
plt.title('Lengths of events')
plt.tight_layout()
plt.show()
#plots track length as a histogram. numberofbins was defined earlier as square root of n
plt.figure(figsize=(5, 3))
plt.hist(redtimes, numberofredbins, color='red')
plt.ylabel('Events')
plt.xlabel('Duration (s)')
plt.title('Lengths of events')
plt.tight_layout()
plt.show()


#histogram of starting positions for each track
redtime = [(track.time_idx[0]) * dt for track in red_tracks.tracks]
##np.array(longest_track.coordinate_idx) * pixel_size / 1000
startpositions = np.array([track.coordinate_idx[0] for track in red_tracks.tracks]) * pixel_size 
plt.figure(figsize=(5, 3))
plt.hist(startpositions, steps_between_bounds, color='red') #Arbitrarily defined a bunch of bins on this, roughly corresponding to 100 nm. This can be altegreen.
plt.xlabel('Start Position (um from top)')
plt.ylabel('Number of events')
plt.tight_layout()
plt.show()


#histogram of mean positions for each track
redmeanpositions = np.array([np.average(track.coordinate_idx) for track in red_tracks.tracks]) * pixel_size
plt.figure(figsize=(5, 3))
plt.hist(redmeanpositions, steps_between_bounds, color='red')
plt.xlabel('Average position (um from top)')
plt.ylabel('Times occuring')
plt.tight_layout()
plt.show()

#histogram of localization precision in um
redstandarddeviation = np.array([np.std(track.coordinate_idx, ddof=1) for track in red_tracks.tracks]) * pixel_size
plt.figure(figsize=(5, 3))
plt.hist(redstandarddeviation, numberofredbins, color='red')
plt.xlabel('Localization precision (um)')
plt.ylabel('Times occuring')
plt.tight_layout()
plt.show()
#this saves the MSD vlues as a .txt file in case we want to graph these in excel/prism. Still under development.
np.savetxt("std.txt", 
           redstandarddeviation,
           delimiter =", ",
            fmt = "%s")

#histogram of normalized track positions in terms of percentage along DNA bound. 
#To function properly, upper and lower bound must be defined as the edge of the beads.
#Any inaccuracies in calling the edge of the beads will also create inaccuracy in this measurement.
rednorm_meanpositions = np.array([np.average(track.coordinate_idx) for track in red_tracks.tracks]) * pixel_size /(adjusted_upper_bound-adjusted_lower_bound)*100
plt.figure(figsize=(5, 3))
plt.hist(rednorm_meanpositions, steps_between_bounds, color='red')
plt.xlabel('Normalized average position (percent from top)')
plt.xlim(0, 100)
plt.ylabel('Times occuring')
plt.tight_layout()
plt.show()

#histogram of the difference between the start and endpoint of each track
reddistancetraveled = np.array([track.coordinate_idx[-1]-track.coordinate_idx[0] for track in red_tracks.tracks]) * pixel_size
plt.figure(figsize=(5, 3))
plt.hist(reddistancetraveled, numberofredbins, color='red')
plt.ylabel('Event number')
plt.xlabel('Distance traveled (um)')
plt.tight_layout()
plt.show()


#histogram of the absolute value of the difference between the start and endpoint of each track
redabsdistancetraveled = np.array([abs(track.coordinate_idx[-1]-track.coordinate_idx[0]) for track in red_tracks.tracks]) * pixel_size
plt.figure(figsize=(5, 3))
plt.hist(redabsdistancetraveled, 11, color='red')
plt.xlabel('Absolute value of distance traveled (um)')
plt.ylabel('Number of events')
plt.tight_layout()
plt.show()


In [ ]:
#green trackS SECTION--------------------------------------------------------------
greenlengths = (track.time_idx[-1]-track.time_idx[0] for track in green_tracks.tracks)
# Get the index of the longest kymo track
greenlongest_index = np.argmax(greenlengths)

# Select the longest green track
greenlongest_track = green_tracks.tracks[greenlongest_index]
dt = kymo.line_time_seconds
greentimes = np.array([(track.time_idx[-1]-track.time_idx[0]) for track in green_tracks.tracks])*dt*downsample_factor

#this section plots the positions of the longest track and also shows the raw image of that track 
#also leaves 1 second empty on each side so the start and end can be seen.
plt.figure(figsize=(5, 3))
plt.plot(np.array(greenlongest_track.time_idx) * dt, np.array(greenlongest_track.coordinate_idx ) * dt, color = 'green') 
plt.xlabel('Time [s]')
plt.ylabel('Position [$\mu$m]')
plt.tight_layout()
plt.show()

plt.figure(figsize=(7, 4))

kymo.plot('rgb', aspect="auto", adjustment=adjustment)
plt.xlim([(np.array(greenlongest_track.time_idx[0])) * dt -1, (np.array(greenlongest_track.time_idx[-1])*dt +1 )])
plt.ylim(adjusted_upper_bound+1, adjusted_lower_bound-1) 
plt.show()

In [ ]:
#THIS CELL FOR PROCESSING green CHANNEL DATA
#to plot photon count along the longest track
plt.figure(figsize=(6, 5))
plt.plot( np.array(greenlongest_track.time_idx) * dt, greenlongest_track.sample_from_image(3), color='green') #this samples 3 pixels away from track position
plt.ylabel('Photon count')
plt.xlabel('Time [s]')
plt.title('Photon counts along the longest track')
plt.tight_layout()
plt.show()


#plots track length as a histogram. numberofbins was defined earlier as square root of n
plt.figure(figsize=(5, 3))
plt.hist(greentimes, numberofgreenbins, color='green')
plt.ylabel('Events')
plt.xlabel('Duration (s)')
plt.title('Lengths of events')
plt.tight_layout()
plt.show()
#plots track length as a histogram. numberofbins was defined earlier as square root of n
plt.figure(figsize=(5, 3))
plt.hist(greentimes, numberofgreenbins, color='green')
plt.ylabel('Events')
plt.xlabel('Duration (s)')
plt.title('Lengths of events')
plt.tight_layout()
plt.show()


#histogram of starting positions for each track
greentime = [(track.time_idx[0]) * dt for track in green_tracks.tracks]
##np.array(longest_track.coordinate_idx) * pixel_size / 1000
startpositions = np.array([track.coordinate_idx[0] for track in green_tracks.tracks]) * pixel_size 
plt.figure(figsize=(5, 3))
plt.hist(startpositions, steps_between_bounds, color='green') #Arbitrarily defined a bunch of bins on this, roughly corresponding to 100 nm. This can be altegreen.
plt.xlabel('Start Position (um from top)')
plt.ylabel('Number of events')
plt.tight_layout()
plt.show()


#histogram of mean positions for each track
greenmeanpositions = np.array([np.average(track.coordinate_idx) for track in green_tracks.tracks]) * pixel_size
plt.figure(figsize=(5, 3))
plt.hist(greenmeanpositions, steps_between_bounds, color='green')
plt.xlabel('Average position (um from top)')
plt.ylabel('Times occuring')
plt.tight_layout()
plt.show()

#histogram of localization precision in um
greenstandarddeviation = np.array([np.std(track.coordinate_idx, ddof=1) for track in green_tracks.tracks]) * pixel_size
plt.figure(figsize=(5, 3))
plt.hist(greenstandarddeviation, numberofgreenbins, color='green')
plt.xlabel('Localization precision (um)')
plt.ylabel('Times occuring')
plt.tight_layout()
plt.show()
#this saves the MSD vlues as a .txt file in case we want to graph these in excel/prism. Still under development.
np.savetxt("std.txt", 
           greenstandarddeviation,
           delimiter =", ",
            fmt = "%s")

#histogram of normalized track positions in terms of percentage along DNA bound. 
#To function properly, upper and lower bound must be defined as the edge of the beads.
#Any inaccuracies in calling the edge of the beads will also create inaccuracy in this measurement.
greennorm_meanpositions = np.array([np.average(track.coordinate_idx) for track in green_tracks.tracks]) * pixel_size /(adjusted_upper_bound-adjusted_lower_bound)*100
plt.figure(figsize=(5, 3))
plt.hist(greennorm_meanpositions, steps_between_bounds, color='green')
plt.xlabel('Normalized average position (percent from top)')
plt.xlim(0, 100)
plt.ylabel('Times occuring')
plt.tight_layout()
plt.show()

#histogram of the difference between the start and endpoint of each track
greendistancetraveled = np.array([track.coordinate_idx[-1]-track.coordinate_idx[0] for track in green_tracks.tracks]) * pixel_size
plt.figure(figsize=(5, 3))
plt.hist(greendistancetraveled, numberofgreenbins, color='green')
plt.ylabel('Event number')
plt.xlabel('Distance traveled (um)')
plt.tight_layout()
plt.show()


#histogram of the absolute value of the difference between the start and endpoint of each track
greenabsdistancetraveled = np.array([abs(track.coordinate_idx[-1]-track.coordinate_idx[0]) for track in green_tracks.tracks]) * pixel_size
plt.figure(figsize=(5, 3))
plt.hist(greenabsdistancetraveled, 11, color='green')
plt.xlabel('Absolute value of distance traveled (um)')
plt.ylabel('Number of events')
plt.tight_layout()
plt.show()


In [ ]:
#plots the photon count for all green tracks, makes a line of best fit and saves this to a txt file and also saves all the plots

# Create a list to store track indices and average photon counts for all green tracks
avg_counts_data = []
figures = []

for i, track in enumerate(green_tracks.tracks):
    # Get time and photon count data
    time_vals = np.array(track.time_idx) * dt
    photon_counts = track.sample_from_image(3)

    # Calculate line of best fit
    coeffs = np.polyfit(time_vals, photon_counts, 1)  # Linear fit (degree 1)
    best_fit_line = np.poly1d(coeffs)

    # Calculate the average y-value
    avg_y = np.mean(photon_counts)
    
    # Store the data for saving in "index, value" format
    avg_counts_data.append(f"{i}, {avg_y:.2f}")

    # Create figure for each plot
    fig, ax = plt.subplots(figsize=(6, 5))
    ax.plot(time_vals, photon_counts, label=f'Track {i}', color='green')
    ax.plot(time_vals, best_fit_line(time_vals), '--', color='black', label='Best Fit')
    ax.set_ylabel('Photon count')
    ax.set_xlabel('Time [s]')
    ax.set_title(f'Photon count for Track {i}\n(Average: {avg_y:.2f})')
    ax.set_ylim(0, 100)
    ax.legend()
    plt.tight_layout()
    
    # Store figure in list
    figures.append(fig)
    
    # Print the average photon count value
    print(f'Track {i}: Average photon count = {avg_y:.2f}')

# Save the average photon counts to a text file
with open("average_green_photon_counts.txt", "w") as file:
    file.write("\n".join(avg_counts_data))

print("Average photon counts saved to average_green_photon_counts.txt")

# Save all figures in a single image file
num_plots = len(figures)
cols = 4  # Number of columns in the grid
rows = (num_plots + cols - 1) // cols  # Compute rows dynamically

fig, axes = plt.subplots(rows, cols, figsize=(cols * 6, rows * 5))
axes = axes.flatten() if num_plots > 1 else [axes]

for ax in axes[len(figures):]:
    ax.axis('off')  # Turn off unused subplots

for i, fig in enumerate(figures):
    fig.canvas.draw()
    img_array = np.array(fig.canvas.renderer.buffer_rgba())
    axes[i].imshow(img_array)
    axes[i].axis('off')
    axes[i].set_title(f'Track {i}')
    plt.close(fig)  # Close individual figures after rendering

plt.tight_layout()
plt.savefig("all_green_photon_counts.png", dpi=300)

# Close the main figure that contains all subplots
plt.close(fig)

print("All plots saved to all_green_photon_counts.png")

In [ ]:
#Colocalization analysis
colocalization_count = 0
interaction_tuple = []
interaction_window = 2 #distance in pixels for an interaction to occur
time_window = 6 #distance in frames for an interaction to occur
#the following loop iterates over the length of each track, comparing every coordinate to the coordinate of all tracks of the other color
#if a point between both is closer than the interaction window and occurs within the time window, the interacting tracks will
#be appended to the interaction tuple as coordinates
for l in range (0, numberofredtracks):
    redtracklength = len((red_tracks.tracks[l].coordinate_idx))
    for i in range (0, redtracklength):
        for j in range (0, numberofgreentracks):
            greentracklength = len((green_tracks.tracks[j].coordinate_idx))
            for k in range (0, greentracklength):
                if abs(green_tracks.tracks[j].coordinate_idx[k]-red_tracks.tracks[l].coordinate_idx[i]) <= interaction_window and abs(green_tracks.tracks[j].time_idx[k] - red_tracks.tracks[l].time_idx[i]) <= time_window: 
                
                    colocalization_count = colocalization_count + 1
                    if len(interaction_tuple) > 0:
                        if j != interaction_tuple [len(interaction_tuple)-1][1]:
                            interaction_tuple.append((l, j))

                            break
                        break
                    else: interaction_tuple.append((l,j))
                break

In [ ]:
#Colocalization analysis
colocalization_count = 0
green_interaction_tuple = []

#the following loop iterates over the length of each track, comparing every coordinate to the coordinate of all tracks of the other color
#if a point between both is closer than the interaction window and occurs within the time window, the interacting tracks will
#be appended to the interaction tuple as coordinates
for l in range (0, numberofgreentracks):
    greentracklength = len((green_tracks.tracks[l].coordinate_idx))
    for i in range (0, greentracklength):
        for j in range (0, numberofredtracks):
            redtracklength = len((red_tracks.tracks[j].coordinate_idx))
            for k in range (0, redtracklength):
                if abs(red_tracks.tracks[j].coordinate_idx[k]-green_tracks.tracks[l].coordinate_idx[i]) <= interaction_window and abs(red_tracks.tracks[j].time_idx[k] - green_tracks.tracks[l].time_idx[i]) <= time_window: 
                
                    colocalization_count = colocalization_count + 1
                    #print (l, j)
                    if len(green_interaction_tuple) > 0:
                        if j != green_interaction_tuple [len(green_interaction_tuple)-1][0]:
                            green_interaction_tuple.append((j, l))

                            break
                        break
                    else: green_interaction_tuple.append((j,l))
                break

            

green_interaction_tuple.sort()     

In [ ]:
unique_sub_cats = []
for ind in green_interaction_tuple:
    if ind not in interaction_tuple:
        unique_sub_cats.append(ind)


for i in range (0, len(unique_sub_cats)):
    interaction_tuple.append(unique_sub_cats[i])

print (interaction_tuple, "are the interacting tracks in the format (red, green)")


In [ ]:
#this window is used for visualizing two interacting tracks. two coordinates from the interaction tuple can be plugged in below
#and both the track position and primary data will be shown side-by-side
plt.close('all')
redtracknumber= 0
greentracknumber = 0



plt.figure(figsize=(7, 4))
ax1 = plt.subplot(2, 1, 1)
kymo.plot("rgb", vmax=1, aspect="auto", adjustment=adjustment)
ax1.set_xlim([(np.array(green_tracks.tracks[greentracknumber].time_idx[0])) * dt -1, (np.array(green_tracks.tracks[greentracknumber].time_idx[-1]) * dt +1 )])
ax1.set_ylim(adjusted_upper_bound+1, adjusted_lower_bound-1) 

ax2 = plt.subplot(2, 1, 2, sharex = ax1, sharey = ax1)
plt.plot(np.array(green_tracks.tracks[greentracknumber].time_idx) * dt, np.array(green_tracks.tracks[greentracknumber].coordinate_idx) * pixel_size + adjusted_lower_bound , color = 'green' ) 
plt.plot(np.array(red_tracks.tracks[redtracknumber].time_idx) * dt, np.array(red_tracks.tracks[redtracknumber].coordinate_idx) * pixel_size + adjusted_lower_bound, color = 'red' ) 
ax2.set_ylim(adjusted_upper_bound+1, adjusted_lower_bound-1)
plt.xlabel('Time [s]')
plt.ylabel('Position [$\mu$m]')
plt.tight_layout()
plt.show()

plt.figure(figsize=(7, 4))
ax1 = plt.subplot(2, 1, 1)
kymo.plot("red", vmax=1, aspect="auto")
ax1.set_xlim([(np.array(red_tracks.tracks[redtracknumber].time_idx[0])) * dt -1, (np.array(red_tracks.tracks[redtracknumber].time_idx[-1]) * dt +1 )])
ax1.set_ylim(adjusted_upper_bound+1, adjusted_lower_bound-1) 

ax2 = plt.subplot(2, 1, 2, sharex = ax1, sharey = ax1)
plt.plot(np.array(green_tracks.tracks[greentracknumber].time_idx) * dt, np.array(green_tracks.tracks[greentracknumber].coordinate_idx) * pixel_size + adjusted_lower_bound, color = 'green' ) 
plt.plot(np.array(red_tracks.tracks[redtracknumber].time_idx) * dt, np.array(red_tracks.tracks[redtracknumber].coordinate_idx) * pixel_size + adjusted_lower_bound, color = 'red' ) 
ax2.set_ylim(adjusted_upper_bound+1, adjusted_lower_bound-1)
plt.xlabel('Time [s]')
plt.ylabel('Position [$\mu$m]')
plt.tight_layout()
plt.show()

In [ ]:
#interaction statistics
total_interactions=len(interaction_tuple)
print (len(interaction_tuple), 'total interactions')
greeninteractions = 1 #there has to be at least one interaction to start with, to compare for unique interactions
if len(interaction_tuple) >0: # this statement checks that there is indeed 1 or more interactions. If not it sets the count to 0
    for i in range (0, len(interaction_tuple)-1):
        if interaction_tuple[i][0] != interaction_tuple[i+1][0]:
            greeninteractions = greeninteractions + 1
else:
    greeninteractions = 0

redinteractions = 1 #same as above but for red tracks
if len(interaction_tuple) >0:
    for k in range (0, len(interaction_tuple)-1):
        if interaction_tuple[k][1] != interaction_tuple[k+1][1]:
            redinteractions = redinteractions + 1
else:
    redinteractions = 0
    



In [ ]:
#further colocalization patterns -- of the colocalizations, how many start with green, how many start with red, how many together?
plt.close('all')
timing_window = 6 #allowed window for events to occur at the same time
interaction_starts = [] #if red comes on first, this is 0, if they start together: 1, if green is first: 2
for i in range (0, len(interaction_tuple)):
    if ((red_tracks.tracks[interaction_tuple[i][0]].time_idx[0]-green_tracks.tracks[interaction_tuple[i][1]].time_idx[0]) > timing_window):
        interaction_starts.append(2)
    if (abs(red_tracks.tracks[interaction_tuple[i][0]].time_idx[0]-green_tracks.tracks[interaction_tuple[i][1]].time_idx[0]) <= timing_window):
        interaction_starts.append(1)
    if ((green_tracks.tracks[interaction_tuple[i][1]].time_idx[0]-red_tracks.tracks[interaction_tuple[i][0]].time_idx[0]) > timing_window):
        interaction_starts.append(0)
#these outputs can be printed out for a double check if desired
#print (interaction_starts)
#print ('From',len(interaction_starts), 'interactions')

#further colocalization patterns -- of the colocalizations, how many end with green, how many end with red, how many together?

timing_window = 6 #allowed window for events to occur at the same time
interaction_ends = [] #if red leaves first, this is 0, if they leave together: 1, if green leaves first: 2
for i in range (0, len(interaction_tuple)):
    if ((red_tracks.tracks[interaction_tuple[i][0]].time_idx[-1]-green_tracks.tracks[interaction_tuple[i][1]].time_idx[-1]) > timing_window):
        interaction_ends.append(2)
    if (abs(red_tracks.tracks[interaction_tuple[i][0]].time_idx[-1]-green_tracks.tracks[interaction_tuple[i][1]].time_idx[-1]) <= timing_window):
        interaction_ends.append(1)
    if ((green_tracks.tracks[interaction_tuple[i][1]].time_idx[-1]-red_tracks.tracks[interaction_tuple[i][0]].time_idx[-1]) > timing_window):
        interaction_ends.append(0)
#these outputs can be printed out for a double check if desired
#print (interaction_ends)
#print ('From',len(interaction_ends), 'interactions')


greenstart = interaction_starts.count(2)
doublestart = interaction_starts.count(1)
redstart = interaction_starts.count(0)


greenend = interaction_ends.count(0)
doubleend = interaction_ends.count(1)
redend = interaction_ends.count(2)
#print (redend)
# this section quantifies these phenomena to make venn diagrams based on what color starts and what color ends
# basically this is a rough way of displaying order of assembly and dissassembly
venn2(subsets = (greenstart, redstart, doublestart), set_labels = ('csb', 'csb'), set_colors=('green', 'red'));
plt.title("Colocalization starting color")
plt.show()
#this section generates a table categorizing each ternary event into 1 of 9 possible categories
# in other words, this is a more rigorous way of showing order of assembly and dissassembly
#this list starts as 3 as category 1 and 2 represent binary events

interaction_categories = []

for i in range (0, len(interaction_tuple)):
    if interaction_starts[i] == 0:
        if interaction_ends[i] == 0:
            interaction_categories.append(11)
        if interaction_ends[i] == 1: 
            interaction_categories.append(10)
        if interaction_ends[i] == 2: 
            interaction_categories.append(9)    
    if interaction_starts[i] == 1:
        if interaction_ends[i] == 0:
            interaction_categories.append(8)
        if interaction_ends[i] == 1: 
            interaction_categories.append(7)
        if interaction_ends[i] == 2: 
            interaction_categories.append(6) 
    if interaction_starts[i] == 2:
        if interaction_ends[i] == 0:
            interaction_categories.append(5)
        if interaction_ends[i] == 1: 
            interaction_categories.append(4)
        if interaction_ends[i] == 2: 
            interaction_categories.append(3)
            
print (interaction_categories)
#this prints out all of the categories as a list

In [ ]:
plt.figure(figsize=(7, 4))
venn2(subsets = (greenend, redend, doubleend), set_labels = ('snm1a', 'csb'), set_colors=('green', 'red'));
plt.title("Colocalization ending color")
plt.show()
#what color ends a colocalization event, i.e. the last color on

In [ ]:
#colocalization lifetimes 
#determines how long interactions occur before one or both partners leaves
# as is this does not account for movement and assumes 1 colocalization for each set of tracks

red_green_colocalization_lifetimes = []
for i in range (0, len(interaction_tuple)):
    if interaction_starts[i] == 0: # this means that red came on first, so the start of the colocalization is the first green time
        coloc_start = green_tracks.tracks[interaction_tuple[i][1]].time_idx[0]
    if interaction_starts[i] == 1: # this means that both colors came on together. For now we'll just take the average start.
        coloc_start = ((green_tracks.tracks[interaction_tuple[i][1]].time_idx[0] + red_tracks.tracks[interaction_tuple[i][0]].time_idx[0])/2)
    if interaction_starts[i] == 2: # this means that green came on first, so the start of the colocalization is the first red time
        coloc_start = red_tracks.tracks[interaction_tuple[i][0]].time_idx[0]
    if interaction_ends[i] == 0: # this means that green left last, so the end of the colocalization is the last red timepoint
        coloc_end = red_tracks.tracks[interaction_tuple[i][0]].time_idx[-1]
    if interaction_ends[i] == 1: # this means that both colors left together. For now we'll just take the average end.
        coloc_end = ((green_tracks.tracks[interaction_tuple[i][1]].time_idx[-1] + red_tracks.tracks[interaction_tuple[i][0]].time_idx[-1])/2)    
    if interaction_ends[i] == 2: # this means that red left last, so the end of the colocalization is the last green timepoint
        coloc_end = green_tracks.tracks[interaction_tuple[i][1]].time_idx[-1]
        
    red_green_colocalization_lifetimes.append((coloc_end-coloc_start) * dt*downsample_factor)
    
print (red_green_colocalization_lifetimes)



In [ ]:
# Load photon counts from the precomputed files
def load_avg_photon_counts(filename):
    avg_counts = {}
    with open(filename, "r") as file:
        for line in file:
            index, value = line.strip().split(", ")
            avg_counts[int(index)] = float(value)
    return avg_counts

# Load average photon counts for red and green tracks
avg_red_counts = load_avg_photon_counts("average_red_photon_counts.txt")
avg_green_counts = load_avg_photon_counts("average_green_photon_counts.txt")

# Your colocalization analysis code to find interaction tuples
colocalization_count = 0
green_interaction_tuple = []

# Colocalization analysis to find interacting red and green tracks
for l in range(0, numberofgreentracks):
    greentracklength = len(green_tracks.tracks[l].coordinate_idx)
    for i in range(0, greentracklength):
        for j in range(0, numberofredtracks):
            redtracklength = len(red_tracks.tracks[j].coordinate_idx)
            for k in range(0, redtracklength):
                if abs(red_tracks.tracks[j].coordinate_idx[k] - green_tracks.tracks[l].coordinate_idx[i]) <= interaction_window and \
                        abs(red_tracks.tracks[j].time_idx[k] - green_tracks.tracks[l].time_idx[i]) <= time_window:
                    colocalization_count += 1
                    if len(green_interaction_tuple) > 0:
                        if j != green_interaction_tuple[len(green_interaction_tuple) - 1][0]:
                            green_interaction_tuple.append((j, l))
                            break
                        break
                    else:
                        green_interaction_tuple.append((j, l))
                break

green_interaction_tuple.sort()

# Merge unique interactions into the interaction_tuple
unique_sub_cats = []
for ind in green_interaction_tuple:
    if ind not in interaction_tuple:
        unique_sub_cats.append(ind)

for i in range(0, len(unique_sub_cats)):
    interaction_tuple.append(unique_sub_cats[i])

print(interaction_tuple, "are the interacting tracks in the format (red, green)")

# Now calculate the start times for red and green tracks that are part of the interaction tuples

# Initialize list to store the output data
output_data = []

# Update downsample_factor
downsample_factor = 1

# Iterate through each interaction tuple and calculate relevant times and values
for red_idx, green_idx in interaction_tuple:
    # Time indices (scaled to seconds)
    red_time_indices = red_tracks.tracks[red_idx].time_idx
    green_time_indices = green_tracks.tracks[green_idx].time_idx

    red_start_time = red_time_indices[0] * dt * downsample_factor
    red_end_time = red_time_indices[-1] * dt * downsample_factor
    red_duration = red_end_time - red_start_time

    green_start_time = green_time_indices[0] * dt * downsample_factor
    green_end_time = green_time_indices[-1] * dt * downsample_factor
    green_duration = green_end_time - green_start_time

    # Calculate overlap time
    overlap_start = max(red_start_time, green_start_time)
    overlap_end = min(red_end_time, green_end_time)
    overlap_time = max(0.0, overlap_end - overlap_start)  # Avoid negative

    # Calculate delay time
    delay_time = green_start_time - red_start_time

    # Retrieve average photon counts
    avg_red_photon_count = avg_red_counts.get(red_idx, "N/A")
    avg_green_photon_count = avg_green_counts.get(green_idx, "N/A")

    # Format and append output
    output_data.append(
        f"{red_idx}, {green_idx}, "
        f"{red_start_time:.2f}, {red_end_time:.2f}, {red_duration:.2f}, "
        f"{green_start_time:.2f}, {green_end_time:.2f}, {green_duration:.2f}, "
        f"{overlap_time:.2f}, {delay_time:.2f}, "
        f"{avg_red_photon_count:.2f}, {avg_green_photon_count:.2f}"
    )

# Save the results to a text file
output_filename = "red_green_track_analysis_with_overlap.txt"
with open(output_filename, "w") as file:
    # Write headers
    file.write("Red Track Index, Green Track Index, "
               "Red Start Time (s), Red End Time (s), Red Duration (s), "
               "Green Start Time (s), Green End Time (s), Green Duration (s), "
               "Overlap Time (s), Delay Time (s), "
               "Avg Red Photon Count, Avg Green Photon Count\n")

    # Write data
    file.write("\n".join(output_data))

print(f"Track analysis (start, end, duration, overlap, photons) saved to {output_filename}")


In [ ]:

#Analysis of single-color events without colocalization only, will output at end with the lengths of all events that didn't colocalize
        

def Filter(list1, list2):  #define a function to filter out any tracks with interactions
    return (list(set(list2) - set(list1)))

green_extracted_list = []
for i in range (0, len(interaction_tuple)):
    green_extracted_list.append(interaction_tuple[i][0])
    
green_track_list = np.arange(0, numberofgreentracks, 1) #this generates a list of green tracks that did not colocalize
solo_green_tracks = Filter(green_extracted_list, green_track_list)



solo_green_times_from_red = []
for k in range (0, len(solo_green_tracks)):
    solo_green_times_from_red.append(greentimes[solo_green_tracks[k]-1])


red_extracted_list = []
for i in range (0, len(interaction_tuple)):
    red_extracted_list.append(interaction_tuple[i][1])
    
red_track_list = np.arange(0, numberofredtracks, 1) #this generates a list of red tracks that did not colocalize
solo_red_tracks = Filter(red_extracted_list, red_track_list)
#filtered = (list(set(red_track_list) - set(red_extracted_list)))



solo_red_times_from_green = []
for k in range (0, len(solo_red_tracks)):
    solo_red_times_from_green.append(redtimes[solo_red_tracks[k]-1])

In [ ]:
#classic venn diagram showing how many red alone, how many green alone, and how many colocalizations
#also lists unique interactions (i.e., how many tracks of a color contribute to the middle portion)

plt.figure(figsize=(7, 4))
venn2(subsets = (len(solo_red_tracks), len(solo_green_tracks), len(red_green_colocalization_lifetimes)), set_labels = ('CSB', 'SNM1A'), set_colors=('red', 'green'), alpha = 0.5) 
plt.show()
print(len(greentimes)-len(solo_green_times_from_red),"unique green interactions")
print(len(redtimes)-len(solo_red_times_from_green),"unique red interactions")
print(len(greentimes))

In [ ]:
#outputs key parameters. Super useful for comparing multiple datasets.
#print (filename[0])

print (median_force)

print (numberofgreentracks)
print (np.floor(max(time)), "s is the final timepoint")
print (dt)
print (pixel_size[0])
#print ('Localization precision is',np.average(standarddeviation)*1000, 'plus or minus', (sem(standarddeviation)*1000), 'nm, assuming no diffusion')

#this section saves down key parameters as .txt files. Can be used with the three-color analysis
np.savetxt("greennorm_meanpositions.txt", 
           greennorm_meanpositions,
           delimiter =", ",
            fmt = "%s")
np.savetxt("rednorm_meanpositions.txt", 
           rednorm_meanpositions,
           delimiter =", ",
            fmt = "%s")
np.savetxt("greentimes.txt", 
           (np.sort (greentimes)),
           delimiter =", ",
            fmt = "%s")
np.savetxt("redtimes.txt", 
           (np.sort (redtimes)),
           delimiter =", ",
            fmt = "%s")
np.savetxt("red_green_colocalization_lifetimes.txt", 
           np.sort(red_green_colocalization_lifetimes),
           delimiter =", ",
            fmt = "%s")
np.savetxt("solo_green_times_from_red.txt", 
           (np.sort (solo_green_times_from_red)),
           delimiter =", ",
            fmt = "%s")
np.savetxt("solo_red_times_from_green.txt", 
           (np.sort (solo_red_times_from_green)),
           delimiter =", ",
            fmt = "%s")

np.savetxt("red_green_interaction_tuple.txt", 
           interaction_tuple,
           delimiter =", ",
            fmt = "%s")

xarray = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
yarray = np.array([len(solo_green_tracks), len(solo_red_tracks), interaction_categories.count(3), interaction_categories.count(4), interaction_categories.count(5), interaction_categories.count(6), interaction_categories.count(7), interaction_categories.count(8), interaction_categories.count(9), interaction_categories.count(10), interaction_categories.count(11)])

data = np.column_stack([xarray, yarray])

np.savetxt("categories_of_colocalization.txt", data, fmt=['%d','%d'])

#this section saves all of the .txt files to a new folder based on colors of analysis
#this way, multiple analyses of the data will not erase each other.
parent_dir = os.getcwd()
directory = "redandgreen_offtarget"
newpath = os.path.join(parent_dir, directory)
os.mkdir(newpath)
path = os.getcwd() 

inpath = path
outpath = (newpath)
file_names = os.listdir(parent_dir)
os.chdir(inpath)
for file in glob.glob("*.txt"):
    shutil.copy(inpath+'/'+file,outpath)

In [ ]:
# Save a file with the average photon counts of the solo green or red timesLoad precomputed average photon counts
def load_avg_photon_counts(filename):
    avg_counts = {}
    with open(filename, "r") as file:
        for line in file:
            index, value = line.strip().split(", ")
            avg_counts[int(index)] = float(value)
    return avg_counts

# Load the photon counts
avg_red_counts = load_avg_photon_counts("average_red_photon_counts.txt")
avg_green_counts = load_avg_photon_counts("average_green_photon_counts.txt")

# Function to filter tracks that didn't colocalize
def Filter(list1, list2):  
    return list(set(list2) - set(list1))

# Identify solo green tracks
green_extracted_list = [interaction[0] for interaction in interaction_tuple]  # Extract green tracks in interactions
green_track_list = np.arange(0, numberofgreentracks, 1)  # All green tracks
solo_green_tracks = Filter(green_extracted_list, green_track_list)

# Identify solo red tracks
red_extracted_list = [interaction[1] for interaction in interaction_tuple]  # Extract red tracks in interactions
red_track_list = np.arange(0, numberofredtracks, 1)  # All red tracks
solo_red_tracks = Filter(red_extracted_list, red_track_list)

# Store output data (Track Index, Avg Photon Count)
green_output_data = [f"{track}, {avg_green_counts.get(track, 'N/A'):.2f}" for track in solo_green_tracks]
red_output_data = [f"{track}, {avg_red_counts.get(track, 'N/A'):.2f}" for track in solo_red_tracks]

# Save the results to text files
with open("solo_green_photoncounts.txt", "w") as file:
    file.write("Track Index, Avg Photon Count\n")
    file.write("\n".join(green_output_data))

with open("solo_red_photoncounts.txt", "w") as file:
    file.write("Track Index, Avg Photon Count\n")
    file.write("\n".join(red_output_data))

print("Solo green photon count data saved to solo_green_photoncounts.txt")
print("Solo red photon count data saved to solo_red_photoncounts.txt")

Happy tracking!

In [ ]:
for i, track in enumerate(red_tracks.tracks):
    lags, msd_values = track.msd(max_lag=1000)
    msd_data = np.column_stack((lags, msd_values))
    
    filename = f"track{i}_msd.txt"
    np.savetxt(filename, 
               msd_data, 
               delimiter=", ", 
               fmt="%.6f", 
               header="Lag, MSD", 
               comments='')

In [ ]:
for i, track in enumerate(green_tracks.tracks):
    lags, msd_values = track.msd(max_lag=1000)
    msd_data = np.column_stack((lags, msd_values))
    
    filename = f"track{i}_msd_green.txt"
    np.savetxt(filename, 
               msd_data, 
               delimiter=", ", 
               fmt="%.6f", 
               header="Lag, MSD", 
               comments='')